In [1]:
!pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122

In [2]:
!huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.2-GGUF mistral-7b-instruct-v0.2.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False

In [ ]:
!git clone https://github.com/raingo/TGIF-Release/

In [ ]:
import pandas as pd
df = pd.read_csv('/kaggle/working/TGIF-Release/data/tgif-v1.0.tsv', sep = '\t', names=['urls','description'])

In [ ]:
def genprompt(inp: str):
    p1 = f"""
    You are a highly qualified expert trained to generate questions based on input text.
    Your task is to analyze the TEXT below and generate ONE question along with its answer that involves the entities, objects, or scenery described in the text.
    The question should strictly focus on the information given in the TEXT. Do not speculate beyond the provided details or context. Your question can be simple or analytical, but it must be directly related to the entities or descriptions found in the TEXT.
    Provide both the question and answer in the following format:
    Question: <your question>
    Answer: <your answer>

    Examples:

    Text: The farmer's market was filled with fresh strawberries, apples, and oranges.
    Question: What fruits were mentioned in the description of the farmer's market?
    Answer: Strawberries, apples, and oranges.

    Text: An old lighthouse stood at the edge of the rocky coastline.
    Question: What structure is mentioned in the text?
    Answer: The structure mentioned in the text is an old lighthouse.

    Text: The bustling city square was adorned with colorful banners and food stalls for the annual festival.
    Question: What event is taking place in the city square?
    Answer: The annual festival is taking place in the city square.

    Now, analyze the following TEXT and generate your question and answer:

    TEXT: {inp}

    Question:
    Answer:
    """
    return p1

In [3]:
from llama_cpp import Llama

llm = Llama(
  model_path="/kaggle/working/mistral-7b-instruct-v0.2.Q4_K_M.gguf", 
  n_ctx=2048,  
  n_threads=8,            
  n_gpu_layers=-1
)

In [ ]:
from tqdm import tqdm
for i in tqdm(range(len(df))):
    prompt = genprompt(df['statement'][i], df['sample_input'][i], df['sample_output'][i])
    try:
        output = llm(
            f'[INST] {prompt} [/INST]',  # Prompt
            max_tokens=1024,  # Generate up to 1024 tokens
            stop=["</s>"],   # Example stop token
            echo=False,       # Whether to echo the prompt
            top_p = 1,
            top_k = 1,
            temperature=0,
        )
        result.append({
          'name' : df['name'][i],
          'year' : df['year'][i],
          'round' : df['round'][i],
          'input' : df['input'][i],
          'output' : df['output'][i],
          'solution' : df['solution'][i],
          'code_ACTUAL' : df['code'][i],
          'statement': df['statement'][i],
          'sample_input': df['sample_input'][i],
          'sample_output': df['sample_output'][i],
          'model_output': output,
      })

    except Exception as e:
        print("weird case ")
        bad.append(i)
    if (i + 1) % 50 == 0:
        checkpoint_data = {
            'result': result,
            'bad': bad,
        }
        with open(f'checkpoint_{i + 1}.json', 'w') as f:
            json.dump(checkpoint_data, f, indent=4)
        print(f"Checkpoint saved at iteration {i + 1}")